In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import time
import datetime as dt
import os
import seaborn as sns
import scipy.stats as sts

import random
import string

import pingouin as pg

import polars as pl

In [ ]:
from nb_vars import FOLDER_INTERMEDIATE, FOLDER_OUTPUT, CUTOFF_SPIKES_HZ

## Load df of filtered peaks

In [ ]:
folder_input =  f'{FOLDER_INTERMEDIATE}'

df_peaks_cutoff = pl.read_csv(f'{folder_input}/df_peaks_full_freq_{CUTOFF_SPIKES_HZ}.csv').to_pandas()


In [ ]:
def adapt_MFR_df(df):
    df['day'] = [1 if '24h' in i else 0 for i in df['time_exp']]
    df.loc[df['time_exp'] == 'Base', 'day'] = -1 
    df['min'] = [int(i.replace('24h + ', '').replace(' min', '')) if i != 'Base' else 0 for i in df['time_exp']]

    df = df.sort_values(by=['day', 'min'])

    df['condition_time'] = df['condition'] + ' | ' + df['time_exp']
    df['condition_time'] = df['condition_time'].apply(lambda x: x.replace(' min', '').replace('h + ', '_'))

    df['time_exp'] = df['time_exp'].apply(lambda x: x.replace(' min', '´'))

    return df

## Relative change in MFR (without Baseline normalisation)

In [ ]:
df_MFR = df_peaks_cutoff.groupby(['condition', 'time_exp', 'well', 'replicate', 'electrode']).count()['time'] / (df_peaks_cutoff['time'].max() - df_peaks_cutoff['time'].min())
df_MFR = df_MFR.reset_index().sort_values(by=['condition', 'time_exp','well', 'electrode'])
df_MFR = df_MFR.rename(columns={'time': 'MFR'})

df_MFR = adapt_MFR_df(df_MFR)
df_MFR

fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.boxplot(data=df_MFR, x='time_exp', y='MFR', hue = 'condition', ax=ax)


In [ ]:
df_MFR

In [ ]:
for tto in ['BLANCO', 'ASTRO PLKO', 'ASTRO 219', 'HEK PLKO', 'HEK 219']:
    fig, ax = plt.subplots(1,1, figsize=(28,6))
    sns.boxplot(data=df_MFR[df_MFR['condition'] == tto], x='time_exp', y='MFR', ax=ax)
    ax.set_title(tto + ' MERGED REPLICATES')
    plt.show()

    fig, ax = plt.subplots(1,1, figsize=(28,6))
    sns.boxplot(data=df_MFR[df_MFR['condition'] == tto], x='time_exp', y='MFR', ax=ax, hue='replicate')
    ax.set_title(tto + ' INDIVIDUAL REPLICATES')
    plt.show()

## Relative change in MFR (with Baseline normalisation)

In [ ]:
def normalize_MFR(df, condition):
    df_sub = df[df['condition'] == condition]
    base_df = df_sub[df_sub['time_exp'] == 'Base']
    rest_df = df_sub[df_sub['time_exp'] != 'Base']

    norm_df = rest_df.copy()
    norm_df = norm_df.rename(columns={'MFR': 'MFR_norm'})

    list_wells_electrodes = base_df[['well', 'electrode']].values

    for well, electrode in zip(list_wells_electrodes.T[0],list_wells_electrodes.T[1]):
        MFR_top = rest_df.loc[(rest_df['well'] == well) & (rest_df['electrode'] == electrode), 'MFR'].values
        MFR_base = base_df.loc[(base_df['well'] == well) & (base_df['electrode'] == electrode), 'MFR'].values

        MFR_norm = (MFR_top / MFR_base)**0.5  # This helps reduce some noise in the data

        norm_df.loc[(norm_df['well'] == well) & (norm_df['electrode'] == electrode), 'MFR_norm'] = MFR_norm

    return norm_df

In [ ]:
df_MFR_norm = normalize_MFR(df_MFR, condition='BLANCO')

fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.boxplot(data=df_MFR_norm, x='time_exp', y='MFR_norm', hue = 'replicate', ax=ax)
ax.set_ylim([0, 2.5])

In [ ]:
df_MFR_norm_list = [normalize_MFR(df_MFR, condition=condition) for condition in df_MFR['condition'].drop_duplicates().values]
df_MFR_norm = pd.concat(df_MFR_norm_list)


fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.boxplot(data=df_MFR_norm, x='time_exp', y='MFR_norm', hue = 'condition', ax=ax)
ax.set_ylim([0, 3])
plt.show()


In [ ]:
for tto in ['BLANCO', 'ASTRO PLKO', 'ASTRO 219', 'HEK PLKO', 'HEK 219']:
    fig, ax = plt.subplots(1,1, figsize=(28,6))
    sns.boxplot(data=df_MFR_norm[df_MFR_norm['condition'] == tto], x='time_exp', y='MFR_norm', ax=ax)
    ax.set_title(tto + ' MERGED REPLICATES')
    plt.show()

    fig, ax = plt.subplots(1,1, figsize=(28,6))
    sns.boxplot(data=df_MFR_norm[df_MFR_norm['condition'] == tto], x='time_exp', y='MFR_norm', ax=ax, hue='replicate')
    ax.set_title(tto + ' INDIVIDUAL REPLICATES')
    plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.boxplot(data=df_MFR_norm[df_MFR_norm['condition'].isin(['BLANCO', 'ASTRO PLKO', 'ASTRO 219'])], x='time_exp', y='MFR_norm',
            hue='condition',  ax=ax, hue_order=['BLANCO', 'ASTRO PLKO', 'ASTRO 219'])
ax.set_title(tto + ' MERGED REPLICATES')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.boxplot(data=df_MFR_norm[df_MFR_norm['condition'].isin(['BLANCO', 'HEK PLKO', 'HEK 219'])], x='time_exp', y='MFR_norm',
            hue='condition',  ax=ax, hue_order=['BLANCO', 'HEK PLKO', 'HEK 219'])
ax.set_title(tto + ' MERGED REPLICATES')
plt.show()

## Double correcting for BLANCO
We are going to calculate a correcting factor for each condition by averaging the value on the control and setting it to 1 in the MFR.

In [ ]:
control_col = 'BLANCO'

df_MFR_norm['MFR_norm_control'] = df_MFR_norm['MFR_norm']

for timepoint in df_MFR_norm['time_exp'].unique():
    df_sub_ctrl = df_MFR_norm[(df_MFR_norm['condition'] == control_col) & (df_MFR_norm['time_exp'] == timepoint)]
    mean_MFR = df_sub_ctrl['MFR_norm'].values.mean()

    df_MFR_norm.loc[df_MFR_norm['time_exp'] == timepoint, 'MFR_norm_control'] /= mean_MFR

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.boxplot(data=df_MFR_norm[df_MFR_norm['condition'].isin(['BLANCO', 'ASTRO PLKO', 'ASTRO 219'])], x='time_exp', y='MFR_norm_control',
            hue='condition',  ax=ax, hue_order=['BLANCO', 'ASTRO PLKO', 'ASTRO 219'])
ax.set_title(tto + ' MERGED REPLICATES')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.lineplot(data=df_MFR_norm[df_MFR_norm['condition'].isin(['BLANCO', 'ASTRO PLKO', 'ASTRO 219'])], x='time_exp', y='MFR_norm_control',
            hue='condition',  ax=ax, hue_order=['BLANCO', 'ASTRO PLKO', 'ASTRO 219'], markers=True, dashes=False, alpha=1, errorbar='sd')
ax.set_title(tto + ' MERGED REPLICATES')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.boxplot(data=df_MFR_norm[df_MFR_norm['condition'].isin(['BLANCO', 'HEK PLKO', 'HEK 219'])], x='time_exp', y='MFR_norm_control',
            hue='condition',  ax=ax, hue_order=['BLANCO', 'HEK PLKO', 'HEK 219'])
ax.set_title(tto + ' MERGED REPLICATES')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(28,6))
sns.lineplot(data=df_MFR_norm[df_MFR_norm['condition'].isin(['BLANCO', 'HEK PLKO', 'HEK 219'])], x='time_exp', y='MFR_norm_control',
            hue='condition',  ax=ax, hue_order=['BLANCO', 'HEK PLKO', 'HEK 219'], errorbar='sd', markers=True, dashes=False, alpha=1)
ax.set_title(tto + ' MERGED REPLICATES')
plt.show()

## Evaluating the conditions with a Mixed Models ANOVA

In [ ]:
# ASTROS


df_MFR_norm_astro = df_MFR_norm[df_MFR_norm['condition'].isin(['BLANCO', 'ASTRO PLKO', 'ASTRO 219'])]
df_MFR_norm_astro['well_electrode'] = df_MFR_norm_astro['well'].astype(str) + ' | ' + df_MFR_norm_astro['electrode'].astype(str)

for col in ["condition", "well_electrode", "replicate"]:
    df_MFR_norm_astro[col] = df_MFR_norm_astro[col].astype("category")

df_MFR_norm_astro["condition"] = df_MFR_norm_astro["condition"].cat.reorder_categories(["BLANCO", "ASTRO PLKO", "ASTRO 219"], ordered=True)


# 
anova_results = pg.mixed_anova(
    dv="MFR_norm_control",
    within="min", 
    between="condition",  
    subject="well_electrode", 
    data=df_MFR_norm_astro
)

display(anova_results)


tukey_results = pg.pairwise_tukey(data=df_MFR_norm_astro, dv="MFR_norm_control", between="condition")
display(tukey_results)


In [ ]:
# HEK

df_MFR_norm_HEK = df_MFR_norm[df_MFR_norm['condition'].isin(['BLANCO', 'HEK PLKO', 'HEK 219'])]
df_MFR_norm_HEK['well_electrode'] = df_MFR_norm_HEK['well'].astype(str) + ' | ' + df_MFR_norm_HEK['electrode'].astype(str)

for col in ["condition", "well_electrode", "replicate"]:
    df_MFR_norm_HEK[col] = df_MFR_norm_HEK[col].astype("category")

df_MFR_norm_HEK["condition"] = df_MFR_norm_HEK["condition"].cat.reorder_categories(['BLANCO', 'HEK PLKO', 'HEK 219'], ordered=True)

# Modelo Lineal Mixto (LMM)
anova_results = pg.mixed_anova(
    dv="MFR_norm_control",
    within="min", 
    between="condition",  
    subject="well_electrode", 
    data=df_MFR_norm_HEK
)

display(anova_results)


tukey_results = pg.pairwise_tukey(data=df_MFR_norm_HEK, dv="MFR_norm_control", between="condition")
display(tukey_results)